# Building a Workspace

 * Build a model Poisson (N | $\mu$ S + B )  with S=10, B=20 constant
 * Generate an observed dataset N=25
 * Construct a unique statistical problem definition (RooStats::ModelConfig) that specifies
 * model = model
 * observable = N
 * parameter of interest = $\mu$
 * Store everything in a root file for later use

Straightforward exploration of the Poisson distribution using RooFit

In [1]:
import ROOT

Welcome to JupyROOT 6.12/06


Construct an empy workspace that will contain the model

In [2]:
w  = ROOT.RooWorkspace("w")


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



## Construct a single Poisson model

Construct expression for expected event count Nexp = mu * S + B, with S and B fixed

In [3]:
w.factory("expr::Nexp('mu*S+B',mu[1,-1,10],S[10],B[20])") 

<ROOT.RooFormulaVar object ("Nexp") at 0x74ed480>

Construct Poisson probability model for Nobs given Nexp

In [4]:
w.factory("Poisson::model(Nobs[0,100],Nexp)")

<ROOT.RooPoisson object ("model") at 0x7809620>

## Construct a dataset containing a single observed event count

Construct an empty dataset for observable Nobs

In [5]:
d = ROOT.RooDataSet("d","d",ROOT.RooArgSet(w.var("Nobs")))

Set Nobs to desired observed event count

In [6]:
w.var("Nobs").setVal(25)

Add observation of 25 events to dataset

In [7]:
d.add(ROOT.RooArgSet(w.var("Nobs")))
getattr(w,'import')(d,ROOT.RooFit.Rename("observed_data"))

False

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing dataset d
[#1] INFO:ObjectHandling -- RooWorkSpace::import(w) changing name of dataset from  d to observed_data


## Construct a unique statistical interpretatation of the model to be used by RooStats

Create an empty ModelConfig

In [8]:
mc = ROOT.RooStats.ModelConfig("ModelConfig",w)

Define the pdf, the parameter of interest and the observables

In [9]:
mc.SetPdf(w.pdf("model"))
mc.SetParametersOfInterest(ROOT.RooArgSet(w.var("mu")))
mc.SetObservables(ROOT.RooArgSet(w.var("Nobs")))

Define the current value $\mu$=1 as an hypothesis

In [10]:
mc.SetSnapshot(ROOT.RooArgSet(w.var("mu")))
mc.Print()


=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (Nobs)
Parameters of Interest:  RooArgSet:: = (mu)
PDF:                     RooPoisson::model[ x=Nobs mean=Nexp ] = 0.0511153
Snapshot:                
  1) 0x7c5e930 RooRealVar:: mu = 1  L(-1 - 10)  "mu"



import model in the workspace 

In [11]:
getattr(w,'import')(mc)
w.writeToFile("../workspaces/model.root")

False